<a href="https://colab.research.google.com/github/PyaesoneGe/sentiment_huggingface/blob/main/sentiment_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:

from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd


In [ ]:
train_data=pd.read_csv("/content/drive/MyDrive/twitter_training.csv")
test_data=pd.read_csv("/content/drive/MyDrive/twitter_validation.csv")

In [ ]:
print(len(train_data))
print(len(test_data))

74681
999


In [ ]:
train_data.columns=['ID','company','sentiment','text']
test_data.columns=['ID','company','sentiment','text']

In [ ]:
train_data = train_data[train_data['sentiment'] != "Irrelevant"]
test_data = test_data[test_data['sentiment'] != "Irrelevant"]

In [ ]:
print(len(train_data))
print(len(test_data))

61691
828


In [ ]:

train_data = train_data[train_data['sentiment'] != "Neutral"]
test_data = test_data[test_data['sentiment'] != "Neutral"]

In [ ]:
print(len(train_data))
print(len(test_data))

43373
543


In [ ]:
train=train_data
test=test_data

In [ ]:
train["text"].isnull().sum()

361

In [ ]:
train.dropna(inplace=True)

In [ ]:
train["text"].isnull().sum()

0

In [ ]:
print(len(train_data))
print(len(test_data))

43012
543


In [ ]:
from tqdm import tqdm

In [ ]:
import re
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def Remove_punctuation(A):
  A['text'] = A['text'].str.replace('[^\w\s]','')


def Remove_stopwords(A):
  stopword = nltk.corpus.stopwords.words('english')
  stop = stopwords.words('english')
  A['text'] = A['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  A.head(20)


def lower_case(A):
  A['text'] = A['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))


def Lemmatization(A):
  nlp = spacy.load("en_core_web_sm")
  def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])
  A['text']= A['text'].apply(space)
  A.head(20)

def re_emoji(A):
  def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
  A['text'] = A['text'].apply(lambda x: remove_emoji(x))

In [ ]:
def data_clean(B):
  lower_case(B)
  Remove_punctuation(B)
  Remove_stopwords(B)
  re_emoji(B)
  Lemmatization(B)

In [ ]:
data_clean(train)

<ipython-input-73-62b190642b57>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  A['text'] = A['text'].str.replace('[^\w\s]','')


In [ ]:
data_clean(test)

<ipython-input-73-62b190642b57>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  A['text'] = A['text'].str.replace('[^\w\s]','')


In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
train['sentiment'] = train['sentiment'].replace(['Positive'], 1)
train['sentiment'] = train['sentiment'].replace(['Negative'], 0)


In [ ]:
test['sentiment'] = test['sentiment'].replace(['Positive'], 1)
test['sentiment'] = test['sentiment'].replace(['Negative'], 0)


In [ ]:
X=list(train['text'])
Y=list(test['text'])

In [ ]:
X_sentiment=list(train['sentiment'])
Y_sentiment=list(test['sentiment'])

In [ ]:
train_encodings = tokenizer(X,
                            truncation=True,
                            padding=True,
                            return_tensors='tf')
test_encodings = tokenizer(Y,
                            truncation=True,
                            padding=True,
                            return_tensors='tf')


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    X_sentiment
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    Y_sentiment
))

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=10,
          batch_size=16,
          validation_data=test_dataset.shuffle(100).batch(16))

Epoch 1/10
2689/2689 [==============================] - 847s 315ms/step - loss: 0.3327 - accuracy: 0.8606 - val_loss: 0.3231 - val_accuracy: 0.8987
Epoch 2/10
2689/2689 [==============================] - 842s 313ms/step - loss: 0.2047 - accuracy: 0.9218 - val_loss: 0.2247 - val_accuracy: 0.9190
Epoch 3/10
2689/2689 [==============================] - 842s 313ms/step - loss: 0.1208 - accuracy: 0.9532 - val_loss: 0.1443 - val_accuracy: 0.9576
Epoch 4/10
2689/2689 [==============================] - 842s 313ms/step - loss: 0.0850 - accuracy: 0.9660 - val_loss: 0.1553 - val_accuracy: 0.9595
Epoch 5/10
2689/2689 [==============================] - 842s 313ms/step - loss: 0.0686 - accuracy: 0.9722 - val_loss: 0.1108 - val_accuracy: 0.9687
Epoch 6/10
2689/2689 [==============================] - 841s 313ms/step - loss: 0.0576 - accuracy: 0.9744 - val_loss: 0.1178 - val_accuracy: 0.9687
Epoch 7/10
2689/2689 [==============================] - 842s 313ms/step - loss: 0.0493 - accuracy: 0.9778 - val_

In [ ]:
model.save_pretrained("/tmp/sentiment_custom_model")

In [ ]:
#### Load saved model and run predict function

In [ ]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("/tmp/sentiment_custom_model")

Some layers from the model checkpoint at /tmp/sentiment_custom_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /tmp/sentiment_custom_model and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
test_sentence = "happy"
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

1/1 [==============================] - 5s 5s/step
[0.347839 0.652161]
1/1 [==============================] - 0s 110ms/step
[0.013792 0.986208]


In [ ]:
model.save_pretrained("/content/drive/MyDrive/model_save")

In [ ]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/model_save")

Some layers from the model checkpoint at /content/drive/MyDrive/model_save were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/model_save and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_sentence = "you are "
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

1/1 [==============================] - 4s 4s/step
[0.990549 0.009451]
1/1 [==============================] - 0s 50ms/step
[0.013792 0.986208]
